In [7]:
import pandas as pd
import numpy as np
import os
import sys
import csv
import glob

In [8]:

def The_Solver(person): 
        data2 = pd.read_excel (person)
        # Select the data containing the team names and results section on the left side columm
        sub1 = data2.iloc[[0,1,2,3,4,5,9,10,11,12,13,14,18,19,20,21,22,23],[2,3]]

        # Give the correct header names to the dataframe
        sub1.columns.values[0] = "homeTeam_goals"
        sub1.columns.values[1] = "awayTeam_goals"


        # Select the data containing the team names and results section on the right side columm
        sub2 = data2.iloc[[0,1,2,3,4,5,9,10,11,12,13,14,18,19,20,21,22,23],[9,10]]

        # Give the correct header names to the dataframe
        sub2.columns.values[0] = "homeTeam_goals"
        sub2.columns.values[1] = "awayTeam_goals"

        # merge the right side table with the left side table to create a simplified dataframe
        data1 = sub1.append(sub2, ignore_index=True,sort=False)

        #Change value types to intergral 

        data1['homeTeam_goals'] = data1['homeTeam_goals'].astype(int)
        data1['awayTeam_goals'] = data1['awayTeam_goals'].astype(int)

        #Generate a vaule difference between home and away score
        data1['Difference'] = data1['homeTeam_goals'] - data1['awayTeam_goals']

        #Create a condition where the difference in goals shows if its a home win, draw and away win
        conditions = [
            (data1['Difference'] < 0),
            (data1['Difference'] == 0),
            (data1['Difference'] > 0)]
        choices = ['Away','Draw','Home']

        #writes value in a new column Result
        data1['Result'] = np.select(conditions, choices, default='null')

        # first data acquired
        # Need to get data from the master sheet
        # Lets get the dataframe from played matches and mix both dataframes
        # Get data from excel spreadsheet located in the folder Euros 2020
        data = pd.read_excel (r'C:\...\Euros2020_test.xlsx')

        # Select the data containing the team names and results section on the left side columm
        test2 = data.iloc[[0,1,2,3,4,5,9,10,11,12,13,14,18,19,20,21,22,23],[1,2,3,4]]

        # Give the correct header names to the dataframe
        test2.columns.values[0] = "home"
        test2.columns.values[1] = "homeTeam_goals0"
        test2.columns.values[2] = "awayTeam_goals0"
        test2.columns.values[3] = "away"

        # Select the data containing the team names and results section on the right side columm
        test3 = data.iloc[[0,1,2,3,4,5,9,10,11,12,13,14,18,19,20,21,22,23],[8,9,10,11]]

        # Give the correct header names to the dataframe
        test3.columns.values[0] = "home"
        test3.columns.values[1] = "homeTeam_goals0"
        test3.columns.values[2] = "awayTeam_goals0"
        test3.columns.values[3] = "away"

        # merge the right side table with the left side table to create a simplified dataframe
        data0 = test2.append(test3, ignore_index=True,sort=False)


        # Merge both dataframes into one
        FinalData = pd.concat([data0, data1.reindex(data0.index)], axis=1)

        #drop NAN
        FinalData1 = FinalData.dropna()

        #get points for exact outcomes
        conditions2 = [((FinalData1['homeTeam_goals0'] == FinalData1['homeTeam_goals']) & (FinalData1['awayTeam_goals0'] == FinalData1['awayTeam_goals'])),
            ((FinalData1['homeTeam_goals0'] != FinalData1['homeTeam_goals']) | (FinalData1['awayTeam_goals0'] != FinalData1['awayTeam_goals'])),
    ]
        choices2 = [2,0]
        FinalData1['points'] = np.select(conditions2, choices2, default='null')

        #get the real results data set to get the overall results of the match 

        conditions3 = [
            ((FinalData1['homeTeam_goals0'] - FinalData1['awayTeam_goals0'])< 0),
            ((FinalData1['homeTeam_goals0'] - FinalData1['awayTeam_goals0']) == 0),
            (FinalData1['homeTeam_goals0'] - FinalData1['awayTeam_goals0']) > 0
            ]


        #writes value in a new column Result
        FinalData1['Real_results'] = np.select(conditions3, choices, default='null')


        #Get points for overall result of the match 
        conditions4 = [
            (FinalData1['Result'] == FinalData1['Real_results']),
            (FinalData1['Result'] != FinalData1['Real_results'])
            ]

        choices3=[1,0]
        #writes value in a new column Result
        FinalData1['points2'] = np.select(conditions4, choices3, default='null')

        #Gives the final result for the contestant 
        Finalpoints = sum(FinalData1['points2'].astype(int)) + sum(FinalData1['points'].astype(int))
        
        #number of correct socres
        Correct_scores = (sum(FinalData1['points'].astype(int)))//2
        
        #Name of the person playing
        person2 = person[:-5]
        

        return [person2,Finalpoints,Correct_scores]
        
    
        
    


In [9]:
#Create a list of files in the directory where the excel files are
A = []
directory = r'C:\...\Predictions'
for filename in os.listdir(directory):
    if filename.endswith(".xlsx"):
        A.append(filename)
    else:
        continue
# create a list where the data takes a excel file, does the magic and gives the person and their score
B = []
for competitor in A:
    B.append(The_Solver(competitor))

#Create a dataframe to manipulate the list and export it as updated_data to the main folder
FinalDF = pd.DataFrame(B,columns =['Names','Scores','Correct Predictions'])
FinalDF.sort_values(['Scores','Correct Predictions'], ascending=(False, False), inplace = True)
FinalDF.to_excel(r'C:\...\update_24_06_21.xlsx', index = False)





In [10]:
The_Solver('Alan Burns.xlsx')

['Alan Burns', 29, 5]